In [1]:
import ee
import geemap
import geopandas as gpd
import numpy as np
import pandas as pd
import spei as si

In [5]:
ee.Authenticate()

Enter verification code:  4/1ATx3LY7Uy5AkxfLzIb0bKb-bx0w_Yp57pdGNRUbW8644AAtdSjLjYWWNQrc



Successfully saved authorization token.


In [8]:
ee.Initialize(project="geo-for-good-ee-python")

In [9]:
dataset = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")

In [10]:
start_date = "2000-01-01"
end_date = "2020-12-31"

In [11]:
terraclimate = dataset.filterDate(start_date, end_date)

In [12]:
precip = terraclimate.select("pr")
pet = terraclimate.select("pet")

In [13]:
spain_provinces = gpd.read_file("../data/maps/spain_map.geojson")

In [14]:
spain_provinces.head()

,community,province,geometry
0,País Vasco/Euskadi,Araba/Álava,"POLYGON ((-2.76808 42.61408, -2.76863 42.61334..."
1,Castilla-La Mancha,Albacete,"POLYGON ((-2.55212 38.08501, -2.55207 38.08517..."
2,Comunitat Valenciana,Alacant/Alicante,"MULTIPOLYGON (((-0.75223 37.88691, -0.75225 37..."
3,Andalucía,Almería,"MULTIPOLYGON (((-3.03624 35.93791, -3.03637 35..."
4,Castilla y León,Ávila,"POLYGON ((-5.43382 40.24491, -5.4343 40.24276,..."


In [25]:
def get_monthly_data(feature):
    province_name = feature["properties"]["province"]
    province_geometry = feature["geometry"]

    # Convert the geometry to an ee.Geometry object
    ee_geometry = ee.Geometry(province_geometry)

    # Aggregate precipitation and PET data over the province
    precip_reduced = (
        precip.mean()
        .reduceRegion(
            reducer=ee.Reducer.mean(), geometry=ee_geometry, scale=1000, maxPixels=1e13
        )
        .getInfo()
    )

    pet_reduced = (
        pet.mean()
        .reduceRegion(
            reducer=ee.Reducer.mean(), geometry=ee_geometry, scale=1000, maxPixels=1e13
        )
        .getInfo()
    )

    return {"province": province_name, "precip": precip_reduced, "pet": pet_reduced}

In [26]:
data = [get_monthly_data(feature) for feature in spain_provinces.iterfeatures()]

In [29]:
df = pd.DataFrame(data)

In [30]:
df

,province,precip,pet
0,Araba/Álava,{'pr': 68.71483612564833},{'pet': 792.6310332466908}
1,Albacete,{'pr': 33.89364474885767},{'pet': 1094.2234909889955}
2,Alacant/Alicante,{'pr': 28.39076935035147},{'pet': 1091.0780625897162}
3,Almería,{'pr': 30.14018937976419},{'pet': 1084.0709587754259}
4,Ávila,{'pr': 45.54843365851541},{'pet': 920.2413715607195}
5,Badajoz,{'pr': 47.81092644950861},{'pet': 1130.4641056755358}
6,Illes Balears,{'pr': 49.26952776161438},{'pet': 977.2442243661221}
7,Barcelona,{'pr': 56.71052972205257},{'pet': 896.7353102350884}
8,Burgos,{'pr': 49.35436872505277},{'pet': 838.4509611534675}
9,Cáceres,{'pr': 50.42425474243369},{'pet': 1067.3006391662693}
